In [28]:
import re

In [29]:
text = "Paul Newman foi um ator estadunidense, mas Paul Hollywood foi um âncora britânico. O nome Paul é muito comum."

In [30]:
pattern = r"Paul [A-Z]\w+" # encontre "Paul " seguido de letra maiúscula [A-Z] e vá até o fim da palavra (\w)

In [31]:
matches = re.finditer(pattern, text)
for match in matches:
    print(match)

<re.Match object; span=(0, 11), match='Paul Newman'>
<re.Match object; span=(43, 57), match='Paul Hollywood'>


In [32]:
import spacy
from spacy.tokens import Span

In [33]:
nlp = spacy.blank("pt")
doc = nlp(text)
original_ents = list(doc.ents)
mwt_ents = []
for match in re.finditer(pattern, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    if span is not None:
        mwt_ents.append((span.start, span.end, span.text))
for ent in mwt_ents:
    start, end, name = ent
    per_ent = Span(doc, start, end, label="PER")
    original_ents.append(per_ent)
doc.ents = original_ents
for ent in doc.ents:
    print(ent.text, ent.label_)

Paul Newman PER
Paul Hollywood PER


In [34]:
print(mwt_ents)

[(0, 2, 'Paul Newman'), (8, 10, 'Paul Hollywood')]


In [35]:
from spacy.language import Language

In [36]:
@Language.component("paul_ner")
def paul_ner(doc):
    pattern = r"Paul [A-Z]\w+"
    original_ents = list(doc.ents)
    mwt_ents = []
    for match in re.finditer(pattern, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end)
        if span is not None:
            mwt_ents.append((span.start, span.end, span.text))
    for ent in mwt_ents:
        start, end, name = ent
        per_ent = Span(doc, start, end, label="PER")
        original_ents.append(per_ent)
    doc.ents = original_ents
    return(doc)  

In [37]:
nlp2 = spacy.blank("pt")
nlp2.add_pipe("paul_ner")

<function __main__.paul_ner(doc)>

In [38]:
doc2 = nlp2(text)
print(doc2.ents)

(Paul Newman, Paul Hollywood)


In [39]:
from spacy.util import filter_spans

@Language.component("cinema_ner")
def cinema_ner(doc):
    pattern = r"Hollywood"
    original_ents = list(doc.ents)
    mwt_ents = []
    for match in re.finditer(pattern, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end)
        if span is not None:
            mwt_ents.append((span.start, span.end, span.text))
    for ent in mwt_ents:
        start, end, name = ent
        per_ent = Span(doc, start, end, label="CINEMA")
        original_ents.append(per_ent)
    filtered = filter_spans(original_ents)
    doc.ents = filtered
    return(doc)  

In [40]:
nlp3 = spacy.load("pt_core_news_sm")
nlp3.add_pipe("cinema_ner")

<function __main__.cinema_ner(doc)>

In [41]:
doc3 = nlp3(text)
for ent in doc3.ents:
    print (ent.text, ent.label_)

Paul Newman PER
Paul Hollywood PER
Paul LOC
